Download some example data from github

In [ ]:
import requests
import tarfile

for filename in ["plikHM_TTTEEE_lowl_lowE_lensing.tar.gz","plikHM_TTTEEE_lowl_lowE_lensing_NS.tar.gz"]:
    github_url = "https://github.com/williamjameshandley/cosmo_example/raw/master/"
    url = github_url + filename
    open('data/' + filename, 'wb').write(requests.get(url).content)
    tarfile.open('data/' + filename).extractall()

This downloaded the PLA chains for the planck baseline cosmology

In [ ]:
import os
os.listdir('data/plikHM_TTTEEE_lowl_lowE_lensing')

And the equivalent nested sampling chains

In [ ]:
os.listdir('data/plikHM_TTTEEE_lowl_lowE_lensing_NS')

Now import anesthetic and load the MCMC samples

In [ ]:
from anesthetic import MCMCSamples
mcmc = MCMCSamples.read('plikHM_TTTEEE_lowl_lowE_lensing/base_plikHM_TTTEEE_lowl_lowE_lensing')

You can see that these are stored as a pandas array

In [ ]:
print(mcmc[:6])

We have plotting tools for 1D plots ...

In [ ]:
fig, axes = mcmc.plot_1d('omegabh2');

... multiple 1D plots ...

In [ ]:
fig, axes = mcmc.plot_1d(['omegabh2','omegach2','H0','tau','logA','ns']);

... triangle plots (with the equivalent scatter plot filling up the left hand side) ...

In [ ]:
mcmc.plot_2d(['omegabh2','omegach2','H0'], types=['kde']);

... triangle plots (with the equivalent scatter plot filling up the left hand side) ...

In [ ]:
mcmc.plot_2d(['omegabh2','omegach2','H0']);

... and rectangle plots.

In [ ]:
mcmc.plot_2d([['omegabh2','omegach2','H0'], ['logA', 'ns']]);

Rectangle plots are pretty flexible with what they can do:

In [ ]:
mcmc.plot_2d([['omegabh2','omegach2','H0'], ['H0','omegach2']]);

More importantly, since this is a pandas array, we can redefine new parameters with relative ease.
 For example, the default cosmoMC setup does not include omegab, only omegabh2:

In [ ]:
'omegab' in mcmc

However, this is pretty trivial to recompute:

In [ ]:
h = mcmc['H0']/100
mcmc['omegab'] = mcmc['omegabh2']/h**2
mcmc.tex['omegab'] = '$\Omega_b$'
mcmc.plot_1d('omegab');

Anethestic really comes to the fore for nested sampling. We can do all of
 the above, and more with the power that NS chains provide

In [ ]:
from anesthetic import NestedSamples
nested = NestedSamples.read('plikHM_TTTEEE_lowl_lowE_lensing_NS/NS_plikHM_TTTEEE_lowl_lowE_lensing')
nested.limits

from anesthetic.read.nested import NestedReader
reader = NestedReader('plikHM_TTTEEE_lowl_lowE_lensing_NS/NS_plikHM_TTTEEE_lowl_lowE_lensing')
reader.limits()

We can infer the evidence, KL divergence and Bayesian model dimensionality:

In [ ]:
ns_output = nested.ns_output(200)
print(ns_output[:6])

This is a set of MCMC samples that may be plotted as usual:

In [ ]:
ns_output.plot_1d();

We can also inspect the correlation between these inferences:

In [ ]:
ns_output.plot_2d(['logZ','D']);

Here is a comparison of the base and NS output

In [ ]:
h = nested['H0']/100
nested['omegab'] = nested['omegabh2']/h**2
nested.tex['omegab'] = '$\Omega_b$'

fig, axes = mcmc.plot_2d(['sigma8','omegab'])
nested.plot_2d(axes=axes);

With nested samples, we can plot the prior (or any temperature), by
 passing beta=0

In [ ]:
from anesthetic import get_legend_proxy

fig, axes = nested.plot_2d(['ns','tau'], beta=0)
nested.plot_2d(axes=axes)
proxy = get_legend_proxy(fig)
fig.legend(proxy, ['prior', 'posterior'])

We can also set up an interactive plot, which allows us to replay a nested
 sampling run after the fact.

In [ ]:
nested.gui()